In [2]:
print('hello pyspark')

hello pyspark


In [3]:
import pandas as pd
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession

In [31]:
spark = SparkSession.builder.appName('SparkAls.com').master('local[*]')\
                .config('spark.driver.memory','10G').config('spark.driver.cores','4')\
                .config('spark.driver.maxResultSize','0').getOrCreate()

spark_context = spark._sc

spark_context.setCheckpointDir('checkpoint/')

In [5]:
review_data = pd.read_json('data_for_spark.json')

In [6]:
user_and_review_count = pd.read_json('user_and_review_count.json')

In [32]:
df_als = spark.createDataFrame(review_data)

In [8]:
df_als.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- beer_beerid: long (nullable = true)
 |-- review_overall: double (nullable = true)



In [9]:
df_als.take(10)

22/12/05 19:59:12 WARN TaskSetManager: Stage 0 contains a task of very large size (6477 KiB). The maximum recommended task size is 1000 KiB.


[Row(user_id=0, beer_beerid=55404, review_overall=4.5),
 Row(user_id=0, beer_beerid=5006, review_overall=4.0),
 Row(user_id=0, beer_beerid=2570, review_overall=3.0),
 Row(user_id=0, beer_beerid=6754, review_overall=3.5),
 Row(user_id=0, beer_beerid=5280, review_overall=3.0),
 Row(user_id=0, beer_beerid=246, review_overall=4.0),
 Row(user_id=0, beer_beerid=69750, review_overall=5.0),
 Row(user_id=0, beer_beerid=4694, review_overall=5.0),
 Row(user_id=0, beer_beerid=4699, review_overall=5.0),
 Row(user_id=0, beer_beerid=31256, review_overall=5.0)]

In [10]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [33]:
(training, test) = df_als.randomSplit([0.8, 0.2])

In [42]:
import time

In [74]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

als = ALS(userCol="user_id", itemCol="beer_beerid", ratingCol="review_overall",
          coldStartStrategy="drop")


paramGrid = ParamGridBuilder()\
     .addGrid(als.regParam, [0.1, 0.01, 0.2, 0.5])\
    .addGrid(als.maxIter, [100])\
    .addGrid(als.alpha, [0.05])\
    .addGrid(als.rank, [10])\
    .addGrid(als.nonnegative, [True])\
     .build()

    
    

tvs = TrainValidationSplit(estimator=als,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(metricName="rmse", labelCol="review_overall",
                                predictionCol="prediction"),
                           trainRatio=0.8)

grid = tvs.fit(df_als)

22/12/05 21:43:50 WARN TaskSetManager: Stage 21804 contains a task of very large size (6477 KiB). The maximum recommended task size is 1000 KiB.
22/12/05 21:43:51 WARN TaskSetManager: Stage 21805 contains a task of very large size (6477 KiB). The maximum recommended task size is 1000 KiB.
22/12/05 21:44:59 WARN TaskSetManager: Stage 22298 contains a task of very large size (6477 KiB). The maximum recommended task size is 1000 KiB.
22/12/05 21:45:02 WARN TaskSetManager: Stage 22330 contains a task of very large size (6477 KiB). The maximum recommended task size is 1000 KiB.
22/12/05 21:45:02 WARN TaskSetManager: Stage 22331 contains a task of very large size (6477 KiB). The maximum recommended task size is 1000 KiB.
22/12/05 21:46:20 WARN TaskSetManager: Stage 22824 contains a task of very large size (6477 KiB). The maximum recommended task size is 1000 KiB.
22/12/05 21:46:21 WARN TaskSetManager: Stage 22856 contains a task of very large size (6477 KiB). The maximum recommended task siz

In [94]:
bestModel = grid.bestModel

In [102]:
bestModel._java_obj.parent().getRegParam()

0.1

In [103]:
als = ALS(maxIter=500, alpha=0.05, rank = 10, nonnegative=True, regParam=0.1, userCol="user_id", itemCol="beer_beerid", ratingCol="review_overall",
          coldStartStrategy="drop")

In [106]:
start = time.time()
model = als.fit(training)
end = time.time()-start
print(start)

22/12/05 22:10:18 WARN TaskSetManager: Stage 26869 contains a task of very large size (6477 KiB). The maximum recommended task size is 1000 KiB.
22/12/05 22:10:19 WARN TaskSetManager: Stage 26870 contains a task of very large size (6477 KiB). The maximum recommended task size is 1000 KiB.


1670278218.0641208


In [134]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="review_overall",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

22/12/05 22:46:11 WARN TaskSetManager: Stage 29585 contains a task of very large size (6477 KiB). The maximum recommended task size is 1000 KiB.


Root-mean-square error = 0.6134694680126662


In [115]:
df_users_als = spark.createDataFrame(user_and_review_count)

In [118]:
df_users_als.take(46)

[Row(user_id=2028, count=451),
 Row(user_id=865, count=451),
 Row(user_id=1281, count=451),
 Row(user_id=2310, count=452),
 Row(user_id=1369, count=452),
 Row(user_id=6048, count=452),
 Row(user_id=3693, count=452),
 Row(user_id=594, count=453),
 Row(user_id=1122, count=453),
 Row(user_id=2579, count=454),
 Row(user_id=2183, count=454),
 Row(user_id=1895, count=454),
 Row(user_id=5861, count=456),
 Row(user_id=3946, count=456),
 Row(user_id=236, count=457),
 Row(user_id=17089, count=3),
 Row(user_id=29201, count=3),
 Row(user_id=29539, count=3),
 Row(user_id=22160, count=3),
 Row(user_id=22221, count=3),
 Row(user_id=10192, count=3),
 Row(user_id=21429, count=3),
 Row(user_id=6544, count=3),
 Row(user_id=12570, count=3),
 Row(user_id=29853, count=3),
 Row(user_id=21435, count=3),
 Row(user_id=6657, count=3),
 Row(user_id=21438, count=3),
 Row(user_id=26310, count=3),
 Row(user_id=9570, count=3),
 Row(user_id=9164, count=46),
 Row(user_id=3401, count=46),
 Row(user_id=11168, count=46),


In [123]:
users = df_users_als.select('user_id')
userSubsetRecs = model.recommendForUserSubset(users, 200)
userSubsetRecs.take(10)

/home/ubuntu/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[Row(user_id=5803, recommendations=[Row(beer_beerid=24961, rating=5.167754173278809), Row(beer_beerid=32744, rating=5.121060848236084), Row(beer_beerid=24864, rating=5.011592388153076), Row(beer_beerid=49042, rating=4.991320610046387), Row(beer_beerid=47661, rating=4.991320610046387), Row(beer_beerid=42941, rating=4.991320610046387), Row(beer_beerid=45400, rating=4.958629131317139), Row(beer_beerid=45680, rating=4.947658538818359), Row(beer_beerid=53731, rating=4.9263153076171875), Row(beer_beerid=76115, rating=4.9263153076171875), Row(beer_beerid=57582, rating=4.90084981918335), Row(beer_beerid=63276, rating=4.895236492156982), Row(beer_beerid=65490, rating=4.895236492156982), Row(beer_beerid=61497, rating=4.892115116119385), Row(beer_beerid=66279, rating=4.891589164733887), Row(beer_beerid=76354, rating=4.884174346923828), Row(beer_beerid=49954, rating=4.880579471588135), Row(beer_beerid=8440, rating=4.867240905761719), Row(beer_beerid=63061, rating=4.8638129234313965), Row(beer_beer

In [125]:
spark_esimations = userSubsetRecs.toPandas()

In [126]:
spark_esimations

,user_id,recommendations
0,5803,"[(24961, 5.167754173278809), (32744, 5.1210608..."
1,29853,"[(8553, 4.585690975189209), (32744, 4.56911849..."
2,7788,"[(54487, 5.1220383644104), (57390, 5.122038364..."
3,236,"[(32744, 5.110413551330566), (24961, 5.0395793..."
4,5478,"[(32744, 6.039188861846924), (24961, 5.9656605..."
5,21429,"[(24961, 5.476594924926758), (21596, 5.3421287..."
6,1122,"[(24961, 5.104242324829102), (32744, 5.0305242..."
7,22160,"[(24961, 4.47920560836792), (57582, 4.46593523..."
8,11168,"[(24961, 5.075491428375244), (63061, 5.0120224..."
9,2183,"[(24961, 5.273130893707275), (32744, 5.1854581..."


In [130]:
spark_esimations.iloc[0]['user_id']

5803

In [136]:
ratings = []
user_ids = []
beer_ids = []

for user in range(len(spark_esimations)):
        for reviews in range(len(spark_esimations.iloc[user]['recommendations'])):
            ratings.append(spark_esimations.iloc[user]['recommendations'][reviews][1])
            beer_ids.append(spark_esimations.iloc[user]['recommendations'][reviews][0])
            user_ids.append(spark_esimations.iloc[user]['user_id'])
            
ratings_export_spark = pd.DataFrame()
ratings_export_spark['ratings'] = ratings
ratings_export_spark['user_id'] = user_ids
ratings_export_spark['beer_id'] = beer_ids

ratings_export_spark[ratings_export_spark['user_id']==0]

ratings_export_spark.to_json('predictions_spark_als.json')

In [124]:
userSubsetRecs.printSchema()

root
 |-- user_id: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- beer_beerid: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)

